In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [12]:

%%time
from storage import Storage
s = Storage()

from html_analysis import HeaderAnalysis
ha = HeaderAnalysis(verbose=True)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

from html_analysis import HeaderCategories
hc = HeaderCategories(cu=cu, verbose=True)

import warnings
warnings.filterwarnings('ignore')


            MATCH (np:NavigableParents {is_task_scope: 'True', is_header: 'True'})
            RETURN np.navigable_parent AS navigable_parent;
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\TASK_SCOPE_HEADERS_LIST.pkl

            MATCH (np:NavigableParents {is_minimum_qualification: 'True', is_header: 'True'})
            RETURN np.navigable_parent AS navigable_parent;
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\REQ_QUALS_HEADERS_LIST.pkl

            MATCH (np:NavigableParents {is_preferred_qualification: 'True', is_header: 'True'})
            RETURN np.navigable_parent AS navigable_parent;
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\PREFF_QUALS_HEADERS_LIST.pkl

            MATCH (np:NavigableParents {is_legal_notification: 'True', is_header: 'True'})
            RETURN np.navigable_parent AS navigable_parent;
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\LEGAL_NO

In [13]:

test_child_strs_list = ['<b>The Role:</b>', "<p>As the Director of Data Science at Brightloom, you'll lead the data science team and " +
                        "partner across the organization to help grow our data science capabilities. You'll be " +
                        "taking business needs from product and engineering teams and creating a comprehensive data " +
                        "science strategy around them. From there, you help prioritize the work for the data science " +
                        "team, balancing the work for the team and helping the independent contributors be their best. " +
                        "You will heavily engage with the application engineering, data engineering, and data science " +
                        "technical leads to ensure that your data science strategies are operationalized effectively.</p>",
                        "<p>You're a person who loves helping others work together to do interesting data science. You " +
                        "can evangelize a data science idea to non-technical people, and you can deal with the " +
                        "realities of a complex business and figure out how to have data science contribute in a " +
                        "positive way. You enjoy leading data scientists so they can do strong work, and helping to " +
                        "elevate the data scientist's concerns when they raise them.</p>", "<b>What you'll do:</b>",
                        "<li>Lead the data science team as their manager - helping devise projects, removing blockers, " +
                        "and engaging with independent contributors. You'll be hiring more data scientists and " +
                        "eventually hire managers to do this part of the job for you.</li>",
                        '<li>Represent the data science discipline throughout the organization. You will have a ' +
                        'powerful voice in the company and represent data scientists across different parts of the ' + 'business.</li>',
                        '<li>Partner across Brightloom engineering groups; evangelizing our culture of using data to ' +
                        'measure, understand, and improve our product and features.</li>',
                        '<li>Be a strong leader - you will give team members clear feedback that helps them grow ' +
                        'and inspires thought leadership. You will lead by example, have compassion for everyone in ' +
                        'the company and their challenges, and will take controversial directions when necessary.</li>',
                        '<li>Be an expert in interpreting data and understanding its conclusions - your team will ' +
                        'be creating lots of powerful insights and you will be finding the valuable implications of ' + 'that data.</li>',
                        '<li>Own the data science technical roadmap - take the business priorities from the product ' +
                        'team and turn them into a set of projects the data science team will focus on delivering.</li>',
                        "<li>Be involved in new product development - as the product team thinks of new ways to take " +
                        "customer feedback and create a better product, you'll be heavily involved with assessing " +
                        "what is feasible from a data science perspective.</li>", '<b>About you:</b>',
                        '<li>Demonstrated experience building and developing a team</li>',
                        '<li>Demonstrated experience putting a data science product into market, ideally in a startup</li>',
                        '<li>Experience in the marketing, retail, or restaurant domain</li>',
                        '<li>Foundational understanding of the data science ecosystem</li>', '<b>About Us</b>',
                        '<p>At Brightloom (formerly eatsa), we are working to revolutionize restaurants through ' +
                        'innovative technology and design. We are disrupting an industry worth $900 billion globally ' +
                        'with partnerships in North America, Asia, and soon other continents.</p>',
                        '<p>Led by our CEO, industry veteran and former Starbucks and J.Crew executive Adam Brotman, our ' +
                        'unique, world-class team combines software and hardware engineers, designers, and industry ' +
                        'experts to push the boundaries on re-engineering every aspect of the restaurant experience.</p>',
                        "<p>We believe any restaurant brand should be able to engage customers digitally using a " +
                        "seamless combination of mobile, omni-channel ordering and loyalty offerings. Up until now, " +
                        "only a select few brands could afford, or knew how to put together a top-notch digital " +
                        "engagement and ordering platform. With key Starbucks technology components integrated into " +
                        "our platform, Brightloom will now allow any restaurant brand to create their own version of a " +
                        "world-class digital flywheel ecosystem. Brightloom's configurable technology suite combines " +
                        "convenience (digital ordering channels), personal connection (personalized marketing) and " +
                        "engagement (loyalty) for restaurant brands in today's new digital era.</p>", '<b>What We Offer</b>',
                        '<li>Fun, creative and collaborative remote work environment</li>', '<li>Competitive pay and equity/stock options</li>',
                        '<li>Health, Dental &amp; Vision Insurance Coverage</li>', '<li>Life Insurance, Short-Term Disability, Long-Term Disability</li>',
                        '<li>Phone/Internet Reimbursement</li>', '<li>Home Office Refresh Reimbursement</li>', '<li>Employee Assistance Program</li>',
                        '<li>Flexible Spending Account &amp; Health Savings Account</li>', '<li>Flexible Time Off</li>', '<li>401(k)</li>',
                        '<p>Brightloom is an Equal Employment Opportunity Employer. All qualified applicants will ' +
                        'receive consideration for employment without regard to race, color, religion, sex, national ' +
                        'origin, sexual orientation, gender identity, disability and protected veterans status or any ' +
                        'other characteristic protected by law.</p>']
pos_list = []
for navigable_parent in test_child_strs_list:
    pos_list = hc.append_parts_of_speech_list(navigable_parent, pos_list)
for navigable_parent, speech_part in zip(test_child_strs_list, pos_list):
    print(f'{speech_part}: {navigable_parent[:100]}')

H-TS: <b>The Role:</b>
O: <p>As the Director of Data Science at Brightloom, you'll lead the data science team and partner acro
O: <p>You're a person who loves helping others work together to do interesting data science. You can ev
H-TS: <b>What you'll do:</b>
O: <li>Lead the data science team as their manager - helping devise projects, removing blockers, and en
O: <li>Represent the data science discipline throughout the organization. You will have a powerful voic
O: <li>Partner across Brightloom engineering groups; evangelizing our culture of using data to measure,
O: <li>Be a strong leader - you will give team members clear feedback that helps them grow and inspires
O: <li>Be an expert in interpreting data and understanding its conclusions - your team will be creating
O: <li>Own the data science technical roadmap - take the business priorities from the product team and 
O: <li>Be involved in new product development - as the product team thinks of new ways to take customer
H-RQ: <b>Abo

In [19]:

HEADER_PATTERN_DICT = s.load_object('HEADER_PATTERN_DICT')
sorted(list(HEADER_PATTERN_DICT.values()), key=lambda x: len(str(x)))[0][0]

{'initial_tag': 'h2', 'is_header': True, 'is_task_scope': True, 'is_minimum_qualification': False, 'is_preferred_qualification': False, 'is_legal_notification': False, 'is_job_title': False, 'is_office_location': False, 'is_job_duration': False, 'is_supplemental_pay': False, 'is_educational_requirement': False, 'is_interview_procedure': False, 'is_corporate_scope': False, 'is_posting_date': False, 'is_other': False, 'child_str': '<h2 class="jobsearch-JobDescriptionSection-jobDescriptionTitle icl-u-xs-my--md" id="jobDescriptionTitle">Full Job Description</h2>'}

In [22]:

child_tags_list = ['b', 'p', 'p', 'b', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'b', 'li', 'li',
                   'li', 'li', 'b', 'p', 'p', 'p', 'b', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'p']

['b', 'p', 'p', 'b', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'b', 'li', 'li', 'li', 'li', 'b', 'p', 'p', 'p', 'b', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'li', 'p']

In [7]:

tag_str = '<b>What We Offer</b>'
print(tag_str)
ha.store_unique_list('SUPP_PAY_HEADERS_LIST', tag_str)
cypher_str = f"""
    MATCH (np:NavigableParents {{navigable_parent: '{tag_str}'}})
    SET np.is_supplemental_pay = 'True', np.is_header = 'True';"""
print(cypher_str)
with cu.driver.session() as session:
    session.write_transaction(cu.do_cypher_tx, cypher_str)

<b>What We Offer</b>
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\SUPP_PAY_HEADERS_LIST.pkl

    MATCH (np:NavigableParents {navigable_parent: '<b>What We Offer</b>'})
    SET np.is_supplemental_pay = 'True', np.is_header = 'True';


In [11]:

tag_str = '<b>About you:</b>'
print(tag_str)
ha.store_unique_list('REQ_QUALS_HEADERS_LIST', tag_str)
cypher_str = f"""
    MATCH (np:NavigableParents {{navigable_parent: '{tag_str}'}})
    SET np.is_minimum_qualification = 'True', np.is_header = 'True';"""
print(cypher_str)
with cu.driver.session() as session:
    session.write_transaction(cu.do_cypher_tx, cypher_str)

<b>About you:</b>
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\REQ_QUALS_HEADERS_LIST.pkl

    MATCH (np:NavigableParents {navigable_parent: '<b>About you:</b>'})
    SET np.is_minimum_qualification = 'True', np.is_header = 'True';


In [5]:

hc.TASK_SCOPE_HEADERS_LIST

['<b>About This Role</b>', '<b>About the Python Software Engineer Position:</b>', '<b>About the</b>', '<b>About this role</b>', '<b>Build our analytics practice:</b>', '<b>Client engagement (25%) :</b>', '<b>Core Responsibilities:</b>', '<b>Core Responsibilities</b>', '<b>DESCRIPTION</b>', '<b>Description</b>', '<b>Duties included but not limited to:</b>', '<b>Employees at all levels are expected to:</b>', "<b>Here's What Data Scientists do at Demandbase:</b>", '<b>JOB DESCRIPTION:</b>', '<b>Job Description:</b>', '<b>Job Description</b>', '<b>Job Duties and Responsibilities</b>', '<b>Job Specification:</b>', '<b>Job Summary:</b>', '<b>Job Summary</b>', '<b>Key Responsibilities:</b>', '<b>Key Responsibilities</b>', '<b>Maintain Webroot’s Machine Learning systems:</b>', '<b>Modeling and Hands-on Analytics (50%):</b>', '<b>OVERVIEW</b>', '<b>Outcomes</b>', '<b>Overview</b>', '<b>Position Description</b>', '<b>Position description</b>', '<b>Primary Responsibilities:</b>', '<b>Principal Du

In [18]:

HEADER_PATTERN_DICT = s.load_object('HEADER_PATTERN_DICT')
list(HEADER_PATTERN_DICT.values())[0][0]

{'initial_tag': 'h2', 'is_header': True, 'is_task_scope': True, 'is_minimum_qualification': False, 'is_preferred_qualification': False, 'is_legal_notification': False, 'is_job_title': False, 'is_office_location': False, 'is_job_duration': False, 'is_supplemental_pay': False, 'is_educational_requirement': False, 'is_interview_procedure': False, 'is_corporate_scope': False, 'is_posting_date': False, 'is_other': False, 'child_str': '<h2 class="jobsearch-JobDescriptionSection-jobDescriptionTitle icl-u-xs-my--md" id="jobDescriptionTitle">Full Job Description</h2>'}

In [4]:

from html_analysis import ElementAnalysis

file_name = '9dc7201db74605e8_Data_Business_Analyst_Remote_Indeed_com.html'
child_strs_list = ha.get_child_strs_from_file(file_name)
ea = ElementAnalysis(ha=ha, hc=None)
ea.find_basic_quals_section(child_strs_list, verbose=True)


            MATCH (pos:PartsOfSpeech)-[r:SUMMARIZES]->(np:NavigableParents)
            RETURN
                np.navigable_parent AS navigable_parent,
                pos.pos_symbol AS pos_symbol;
pos_df.columns = Index(['navigable_parent', 'pos_symbol'], dtype='object')
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

([('O', 15), ('H-TS', 1), ('O', 15), ('H-RQ', 1), ('O', 13), ('H-SP', 1), ('O', 29)], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'H-TS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'H-RQ', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'H-SP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])

In [13]:

cypher_str = '''
    MATCH (pos:PartsOfSpeech)
    UNWIND pos.pos_symbol as s
    RETURN count(DISTINCT s) AS num_topics;'''
row_objs_list = cu.get_execution_results(cypher_str, verbose=True)
row_objs_list[0]['num_topics']


    MATCH (pos:PartsOfSpeech)
    UNWIND pos.pos_symbol as s
    RETURN count(DISTINCT s) AS num_topics;


26

In [5]:

ea.display_basic_requirements(child_strs_list)

In [ ]:

from html_analysis import LrUtilities

lu = LrUtilities(ha=ha, hc=None, cu=cu, verbose=False)
predict_percent_fit = lu.build_lr_predict_percent_is_header()

In [23]:

from sklearn.feature_extraction.text import CountVectorizer

if s.pickle_exists('CS_CV'):
    lu.CS_CV = s.load_object('CS_CV')
else:
    lu.CS_CV = CountVectorizer(
        analyzer='word',
        binary=False,
        decode_error='strict',
        lowercase=False,
        max_df=1.0,
        max_features=None,
        min_df=0.0,
        ngram_range=(1,5),
        stop_words=None,
        strip_accents='ascii',
        tokenizer=ha.html_regex_tokenizer)
    s.store_objects(CS_CV=lu.CS_CV)

In [25]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression

# Re-transform the bag-of-words and tf-idf from the new manual scores
rows_list = [{'navigable_parent': navigable_parent,
              'is_header': is_header} for navigable_parent, is_header in ha.NAVIGABLE_PARENT_IS_HEADER_DICT.items()]
child_str_df = pd.DataFrame(rows_list)
child_str_df.is_header = child_str_df.is_header.astype('bool')

# The shape of the Bag-of-words count vector here should be n sentences * m unique words
sents_list = child_str_df.navigable_parent.tolist()
bow_matrix = lu.CS_CV.fit_transform(sents_list)

# Tf-idf must get from Bag-of-words first
tfidf_matrix = lu.TT.fit_transform(bow_matrix)

In [35]:

# Re-train the classifier
try:
    import gc

    gc.collect()
    X = tfidf_matrix.toarray()
except Exception as e:
    print(f'{e.__class__} error: {str(e).strip()}')
    X = tfidf_matrix

<class 'numpy.core._exceptions._ArrayMemoryError'> error: Unable to allocate 62.0 GiB for an array with shape (11712, 710081) and data type float64


In [19]:

if s.pickle_exists('CHILD_STR_CLF'):
    lu.CHILD_STR_CLF = s.load_object('CHILD_STR_CLF')
else:
    lu.CHILD_STR_CLF = LogisticRegression(
        C=375.0,
        class_weight='balanced',
        dual=False,
        fit_intercept=True,
        intercept_scaling=1,
        l1_ratio=None,
        max_iter=1000,
        multi_class='auto',
        n_jobs=None,
        penalty='l1',
        random_state=None,
        solver='liblinear',
        tol=0.0001,
        verbose=0,
        warm_start=False)
    s.store_objects(CHILD_STR_CLF=lu.CHILD_STR_CLF)

In [26]:

y = child_str_df.is_header.to_numpy()
lu.CHILD_STR_CLF.fit(tfidf_matrix, y)

LogisticRegression(C=10.0, class_weight='balanced', max_iter=1000, penalty='l1',
                   solver='liblinear')

In [27]:

s.store_objects(CHILD_STR_CLF=lu.CHILD_STR_CLF)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\CHILD_STR_CLF.pkl


In [28]:

# Re-calibrate the inference engine
cv = CountVectorizer(vocabulary=lu.CS_CV.vocabulary_)
cv._validate_vocabulary()

def predict_percent_fit(navigable_parent):
    X_test = lu.TT.transform(cv.transform([navigable_parent])).toarray()
    y_predict_proba = lu.CHILD_STR_CLF.predict_proba(X_test)[0][1]

    return y_predict_proba

In [31]:

child_str_df['predict_percent_fit'] = child_str_df.navigable_parent.map(lambda x: predict_percent_fit(x))

In [34]:

mask_series = child_str_df.is_header
max_false_percent_fit = child_str_df[~mask_series].predict_percent_fit.max()
print(max_false_percent_fit)
display(child_str_df[~mask_series].sort_values('predict_percent_fit', ascending=False))
min_true_percent_fit = child_str_df[mask_series].predict_percent_fit.min()
print(min_true_percent_fit)
display(child_str_df[mask_series].sort_values('predict_percent_fit', ascending=True))

0.9999999999944673


,navigable_parent,is_header,predict_percent_fit
1498,<b>will not</b>,False,1.000000e+00
648,<b>Great analytics</b>,False,1.000000e+00
10937,"<li>Telerik &amp; Kendo UI Tools, NodeJS, Angu...",False,1.000000e+00
138,. See below for more information.,False,1.000000e+00
979,<b>Python &amp; AIMMS;</b>,False,1.000000e+00
...,...,...,...
1487,<b>or</b>,False,4.271966e-18
863,<b>OR</b>,False,4.271966e-18
249,<b>5+ years</b>,False,4.162431e-21
53,*: 3-6 years,False,4.162431e-21


3.0143009290415106e-07


,navigable_parent,is_header,predict_percent_fit
250,<b>6 years of analytics experience:</b>,True,3.014301e-07
380,<b>Big Data:</b>,True,2.746350e-06
409,<b>Careers with Optum.</b>,True,6.361448e-06
9574,Collaborate with the:,True,2.182146e-05
480,<b>Data storyteller:</b>,True,7.441793e-05
...,...,...,...
1432,<b>You will:</b>,True,1.000000e+00
1435,<b>You:</b>,True,1.000000e+00
349,<b>At Compass You Will:</b>,True,1.000000e+00
1422,<b>You Will:</b>,True,1.000000e+00
